## Contextualized model

Let's check where the context really helps

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
%load_ext autoreload
%autoreload 2

from hatedetection import load_datasets

train_dataset, dev_dataset, test_dataset = load_datasets(add_body=True)


In [3]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

context_model_name = "../models/bert-contextualized-hate-speech-es/"
no_context_model_name = "../models/bert-non-contextualized-hate-speech-es"
full_context_model_name = "../models/bert-title-body-hate-speech-es/"


context_model = AutoModelForSequenceClassification.from_pretrained(context_model_name, return_dict=True, num_labels=2)
no_context_model = AutoModelForSequenceClassification.from_pretrained(no_context_model_name, return_dict=True, num_labels=2)
full_context_model = AutoModelForSequenceClassification.from_pretrained(no_context_model_name, return_dict=True, num_labels=2)

context_model.eval()
no_context_model.eval()
full_context_model.eval()


device = "cuda" if torch.cuda.is_available() else "cpu"

context_model = context_model.to(device)
no_context_model = no_context_model.to(device)
full_context_model = full_context_model.to(device)
# Tienen mismo tokenizer así que todo bien

no_context_tokenizer = AutoTokenizer.from_pretrained(no_context_model_name)
context_tokenizer = AutoTokenizer.from_pretrained(context_model_name)
full_context_tokenizer = AutoTokenizer.from_pretrained(full_context_model_name)

no_context_tokenizer.model_max_length = 128
context_tokenizer.model_max_length = 256

In [4]:
from hatedetection.training import tokenize

batch_size = 32
eval_batch_size = 16

dataset = test_dataset


no_context_dataset = dataset.map(lambda x: tokenize(no_context_tokenizer, x, context='none'), batched=True, batch_size=eval_batch_size)
context_dataset = dataset.map(lambda x: tokenize(context_tokenizer, x, context='title'), batched=True, batch_size=eval_batch_size)
full_context_dataset = dataset.map(lambda x: tokenize(full_context_tokenizer, x, context='title+body'), batched=True, batch_size=eval_batch_size)


In [5]:

def format_dataset(dataset):
    dataset = dataset.map(lambda examples: {'labels': examples['HATEFUL']})
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    return dataset

no_context_dataset = format_dataset(no_context_dataset)
context_dataset = format_dataset(context_dataset)
full_context_dataset = format_dataset(full_context_dataset)


Lo cargamos sólo para evaluar 🤗

In [6]:
from hatedetection.metrics import compute_hate_metrics
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=".",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
)


context_trainer = Trainer(
    model=context_model,
    args=training_args,
    compute_metrics=compute_hate_metrics,
)

full_context_trainer = Trainer(
    model=full_context_model,
    args=training_args,
    compute_metrics=compute_hate_metrics,
)

no_context_trainer = Trainer(
    model=no_context_model,
    args=training_args,
    compute_metrics=compute_hate_metrics,
)


## Resultados

In [7]:
import pandas as pd
pd.options.display.max_columns = 40
pd.set_option('display.float_format', lambda x: '%.5f' % x)


df_full_context_results = pd.DataFrame([full_context_trainer.evaluate(full_context_dataset)])
df_context_results = pd.DataFrame([context_trainer.evaluate(context_dataset)])
df_no_context_results = pd.DataFrame([no_context_trainer.evaluate(no_context_dataset)])


In [8]:
df_results = pd.concat([df_no_context_results, df_context_results, df_full_context_results])
df_results["index"] = ["No context", "Title", "Title and Body"]
df_results.set_index("index")

,eval_loss,eval_accuracy,eval_f1,eval_macro_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,init_mem_cpu_alloc_delta,init_mem_gpu_alloc_delta,init_mem_cpu_peaked_delta,init_mem_gpu_peaked_delta,eval_mem_cpu_alloc_delta,eval_mem_gpu_alloc_delta,eval_mem_cpu_peaked_delta,eval_mem_gpu_peaked_delta
index,,,,,,,,,,,,,,,,
No context,0.65833,0.88945,0.63525,0.78505,0.66545,0.60768,71.59330,158.43700,43460,0,18258,0,233460,0,897173,69492736
Title,0.80262,0.90505,0.66563,0.80515,0.75281,0.59655,105.10900,107.91600,52521,0,18258,0,247803,0,897045,189087744
Title and Body,0.81926,0.87137,0.35585,0.64220,0.86111,0.22426,286.39620,39.60600,43213,0,18258,0,501766,0,887205,579272704


## Dev Results

In [8]:
df_results = pd.concat([df_no_context_results, df_context_results, df_full_context_results])
df_results["index"] = ["No context", "Title", "Title and Body"]
df_results.set_index("index")

,eval_loss,eval_accuracy,eval_f1,eval_macro_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,init_mem_cpu_alloc_delta,init_mem_gpu_alloc_delta,init_mem_cpu_peaked_delta,init_mem_gpu_peaked_delta,eval_mem_cpu_alloc_delta,eval_mem_gpu_alloc_delta,eval_mem_cpu_peaked_delta,eval_mem_gpu_peaked_delta
index,,,,,,,,,,,,,,,,
No context,0.50128,0.90753,0.66980,0.80802,0.73431,0.61572,38.45080,236.82200,43460,0,18258,0,200737,0,755193,69447680
Title,0.60040,0.92554,0.73883,0.84771,0.79322,0.69142,76.05850,119.72400,52465,0,18258,0,209357,0,756777,189042688
Title and Body,0.85207,0.86130,0.18778,0.55598,0.86905,0.10526,156.20100,58.29700,43325,0,18258,0,430762,0,742281,579227648


In [19]:
example = context_dataset[0]

context_tokenizer.decode(example["input_ids"])

'[CLS] [USER] ojalá se funda y cague de hambre [SEP] mora godoy cierra su escuela de tango y remata su vestuario para " poder seguir adelante " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD